Preprocessing - Scaling all variables

In [ ]:
from sklearn.preprocessing import MinMaxScaler
df = pd.DataFrame({'b':range(1,5),'c':range(11,15)})

# Convert the dataframe to matrix and scale
X_train = MinMaxScaler().fit_transform(pd.DataFrame(df).as_matrix()) # Creates a object of the minmaxscaler class
print pd.DataFrame(df).as_matrix()         # whose function fit_transform we use to transform the data
print X_train                              # from a matrix to a scaled matrix vertically (column wise)

                                           # In case of single col you need to conver from 1-D array to 
                                           # column vector before using fit_transform as it always takes a 2-D array  

General Code for running multiple models, outputting AUC curve, accuracy and variable importance values

In [ ]:
# build the model for test data
gbc = GradientBoostingClassifier(random_state=123)
rf = RandomForestClassifier(n_estimators=30,max_depth=10,random_state=123)
lr = LogisticRegression(penalty='l1',tol=0.01)
# plot the ROC curve
%matplotlib inline
plt.figure()
plt.plot([0,1],[0,1],color='navy',lw=2,linestyle='--')
plt.xlim([0,1])
plt.ylim([0,1.05])
for model,name in [(gbc,'Gradient Boosting Classifier'),(rf,'Random Forest'),(lr,'Lasso Logistic Regression')]:
    model.fit(X_train,Y_train,sample_weight=weight_train)
    Y_predict = model.predict(X_test)
    Y_predict_prob = model.predict_proba(X_test)[:,1]
    # print the accuracy of the prediction
    print "The accuracy score of " + name + " is: " + str(model.score(X_test,Y_test,sample_weight=weight_test))
    fpr,tpr,thresholds = metrics.roc_curve(Y_test,Y_predict_prob,pos_label=1,sample_weight=weight_test)
    plt.plot(fpr,tpr,lw=2,label=name)
    # obtain the ROC score
    print "The AUC area of " + name + " is: " + str(metrics.roc_auc_score(Y_test,Y_predict_prob,sample_weight=weight_test))
   
    if name == 'Lasso Logistic Regression':
        Y_predict_prob_lr = Y_predict_prob
        print "The variable importance from Lasso Logistic Regression:"
        # get the coefficient of the linear regression
        coeff = model.coef_.flatten().tolist()
        coeff = map(abs,coeff)
        coeff_dict={}
        for i in range(len(coeff)):
            coeff_dict[cols_name[i]]=coeff[i]
        sorted_coeff_dict = sorted(coeff_dict.items(),key=operator.itemgetter((1)),reverse=True)
#         for x in sorted_coeff_dict:
#            print x
    if name == 'Random Forest':
        Y_predict_prob_rf = Y_predict_prob
        print "The variable importance from Random Forest:"
        coeff = model.feature_importances_.flatten().tolist()
        coeff = map(abs,coeff)
        coeff_dict={}
        for i in range(len(coeff)):
            coeff_dict[cols_name[i]]=coeff[i]
        sorted_coeff_dict = sorted(coeff_dict.items(),key=operator.itemgetter((1)),reverse=True)
        for x in sorted_coeff_dict:
            print x
    if name == 'Gradient Boosting Classifier':
        Y_predict_prob_gbc = Y_predict_prob
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic graph')
plt.legend(loc="lower right")